In [ ]:
import numpy as np
import pandas as pd 
from sklearn.neighbors import BallTree
from tqdm import tqdm

This notebook was loosely inspired by Heng Zheng's notbebook:

https://www.kaggle.com/code/hengzheng/distance-and-category-hard-match-baseline/notebook?scriptVersionId=93093506

Which is based on pjmathematician's notebook:

https://www.kaggle.com/code/pjmathematician/matching-based-on-nearest-location

In [ ]:
#train = pd.read_csv('../input/foursquare-location-matching/train.csv')
test = pd.read_csv('../input/foursquare-location-matching/test.csv')
sample_submission = pd.read_csv('../input/foursquare-location-matching/sample_submission.csv')

In [ ]:
#train_df.categories = train_df.categories.fillna('__NAN__')
test.categories = test.categories.fillna('__NAN__')

In [ ]:
tree = BallTree(np.deg2rad(test[['latitude', 'longitude']].values), metric='haversine')

In [ ]:
# list for storing the points of interest
pois_out = []
# number of neighbours considered
n = min(10, len(test))
# max number of recommended points of interest
max_poi = 2
# max distance
max_dist = 0.0005

for i, row in tqdm(test.iterrows()):
    dist, ind = tree.query(np.deg2rad(np.c_[row['latitude'], row['longitude']]), k = n)
    poi = []
    for d, j in zip(dist[0], ind[0]):
        if d <= max_dist and row['categories'] != '__NAN__' and (row['categories'] in test.categories.iloc[j] or test.categories.iloc[j] in row['categories']):
            poi.append(test.id.iloc[j])
        if d > max_dist or len(poi) >= max_poi:
            break
    if len(poi) == 0:
        pois_out.append(row['id'])
    else:
        pois_out.append(' '.join(poi))

In [ ]:
sample_submission.matches = pois_out
sample_submission.head()

In [ ]:
sample_submission.to_csv('submission.csv', index=False)